# Upload Audience Data to BigQuery

In [1]:
%load_ext autoreload
%autoreload 2

::: {.content-hidden}
Import necessary Python modules
:::

In [2]:
import os
import sys
from calendar import day_name, month_name
from datetime import datetime
from glob import glob

import pandas as pd
import pytz
from google.cloud import bigquery
from google.oauth2 import service_account

::: {.content-hidden}
Get relative path to project root directory
:::

In [3]:
PROJ_ROOT_DIR = os.path.join(os.pardir)
src_dir = os.path.join(PROJ_ROOT_DIR, "src")
sys.path.append(src_dir)

::: {.content-hidden}
Import custom Python modules
:::

In [4]:
%aimport bigquery_auth_helpers
from bigquery_auth_helpers import auth_to_bigquery

%aimport bigquery_upload_helpers
import bigquery_upload_helpers as bquh

%aimport sql_helpers
import sql_helpers as sqlh

%aimport transform_helpers
import transform_helpers as trh

::: {.content-hidden}
Define helper function to show datatypes and number of missing values for all columns in a `DataFrame`
:::

In [5]:
def summarize_df(df: pd.DataFrame) -> None:
    """Show datatypes and count missing values in columns of DataFrame."""
    display(
        df.dtypes.rename("dtype")
        .to_frame()
        .merge(
            df.isna().sum().rename("missing").to_frame(),
            left_index=True,
            right_index=True,
            how="left",
        )
        .reset_index()
        .rename(columns={"index": "column"})
    )

## About

## Overview

This step will upload the following to separate tables in a private dataset on Google BigQuery

1. predicted audience groups and (test and control) cohorts<sup>[1](#myfootnote1)</sup>
2. audience profiles<sup>[1](#myfootnote1)</sup>
3. monthly performance<sup>[1](#myfootnote1)</sup>, relative to previous months
4. summary of required sample (cohort) sizes that were estimated in a previous step and metrics per cohort, based on desired inputs (combination of uplift, power, confidence level)

These tables will be accessed by the client-facing dashboard.

<a name="myfootnote1">1</a>: using inference data

## Order of Operations
This step can be run prospectively at the end of the inference period, just before the start of the campaign, when all the inference data (first-time visitors to the store) becomes available. This step is needed before the project's dashboard (main project deliverable) can be created since the dashboard will be populated with the data derived from data uploaded to BigQuery tables in this step.

## Outputs
The following tables will be created

1. Combined Audience Profile and Feature Importances
   - audience profile consisting of the following for each predicted audience group
     - audience strategy
     - name of descriptive or behavioral statistic about attribute of first visit
     - value of statistic in
       - High propensity group of visitors
       - Medium propensity group of visitors
       - Low propensity group of visitors
   - ML feature importances
     - audience strategy
     - number of observations used to learn feature importances
     - feature name
     - audience group
     - feature importance
2. All Available First-Time Visitors and Assigned Audience Groups and Cohorts
   - month during which first-time visitors visited the store site during inference period
     - for development data, no audience or cohorts are assigned
     - for inference predictions, audience groups and randomly selected cohorts are assigned
   - visitor ID from GA360 data
   - visit ID from GA360 data
   - visit number from GA360 data
   - visit start time
   - quarter of year during which visit occurred
   - month of year during which visit occurred
   - day of month during which visit occurred
   - day of week during which visit occurred
   - hour of day during which visit occurred
   - minute of hour during which visit occurred
   - second of minute during which visit occurred
   - traffic source from GA360 data
   - medium from GA360 data
   - channel from GA360 data
   - registered hits from GA360 data
   - bounced visits from GA360 data
   - last action performed during visit from GA360 data
   - number of promotions viewed from GA360 data
   - number of promotions clicked from GA360 data
   - number of product lists viewed from GA360 data
   - number of products clicked from GA360 data
   - pageviews from GA360 data
   - time spent on store site from GA360 data
   - browser used to access store site from GA360 data
   - operating system used to access store site from GA360 data
   - category of device used to access store site from GA360 data
   - whether a product was added to the shopping cart during first visit from GA360 data
   - revenue during first visit from GA360 data
   - predicted propensity for making a purchase during a return visit
   - predicted audience group
   - predicted cohort (test or control)
   - audience strategy (single- or multiple-audience)
   - true outcome (whether a purchase was made on a return visit to the store)
     - missing for inference data
   - type of data (development - `train_val`/`test` - or inference - `infer`)
4. Monthly Summary Statistics (Overall)
   - month during which first-time visitors visited the store site during inference period
   - type of data (development - `train_val`/`test` - or inference - `infer`)
   - total number of return purchasers
   - total revenue
   - total number of first-time visitors
   - total pageviews
   - average time spent on the store site
   - most populat channel
   - most popular category of device
   - most popular browser
   - most popular operating system
   - audience strategy (single- or multiple-audience)
   - bounce rate
   - product click rate
   - add-to-cart rate
   - type of visitor (development or inference)
   - conversion rate
   - rate of change in the following relative to the previous month
     - total number of return purchasers (development) or first-time visitors (inference)
     - total revenue
     - total pageviews
     - average time spent on the store site
     - bounce rate
     - conversion rate
     - product click rate
     - add-to-cart rate
5. Conversion Rates During Development and Inference
   - over the period covering the development and inference data
     - audience strategy (single- or multiple-audience)
     - month during which first-time visitors visited the store site during inference period
     - audience group (eg. High, Medium, Low predicted propensity)
     - predicted conversions
     - total number of visitors per audience group
     - minimum score
     - true number of conversions per audience group
     - type of visitor (development or inference)
     - total number of each type of visitor
     - true conversion rate per audience group
     - overall true conversion rate
     - predicted conversion rate per audience group
     - overall predicted conversion rate
6. Estimated and Actual Fractions of Cohort to Audience Size
   - over the period covering the development and inference data
     - audience strategy (single- or multiple-audience)
     - month during which first-time visitors visited the store site during inference period
     - audience group (eg. High, Medium, Low predicted propensity)
     - cohort (test or control)
     - cohort size
     - audience size
     - required uplift, power and confidence level used to estimate cohort size
     - ratio of cohort to audience sizes
     - type of cohort size
       - development (estimated based on required uplift, power and confidence level)
       - inference (assigned using estimates during development)
     - type of visitor (development or inference)
     - total number of each type of visitor
7. Aggregated Conversion Rates During Development and Inference (by Audience Group & Overall)
   - over the period covering the development and inference data
     - audience group (eg. High, Medium, Low predicted propensity)
     - data_type (development or inference)
     - true or predicted conversion rate
8. Daily Summary Statistics by Audience Group and Overall
   - over the period covering the development and inference data
     - total number of return purchasers
     - total revenue
     - total number of first-time visitors
     - total number of add-to-cart actions performed
     - total pageviews
     - average time spent on the store site
     - total product lists viewed
     - total products clicked on
     - total number of bounce events
     - audience strategy (single- or multiple-audience)
     - bounce rate
     - product click rate
     - add-to-cart rate
     - type of visitor (development or inference)
     - aggregation type (by audience group or overall)
9. KPIs by categorical feature
   - over the period covering the test split of the development data, show the following by sub-category
     - categorical feature
     - categorical feature value (sub-category)
     - number of conversions
     - number of visitors
     - average conversion rate
     - average clickthrough rate
     - total number of conversions
     - total number of clicks
     - total product lists viewed
     - total products clicked on
     - total revenue
     - total number of visitors 

## User Inputs

Define the following

1. best MLFlow run ID
2. list of categorical features
3. list of numerical features
4. BigQuery
   - dataset id
   - table ids for audience
     - cohorts
     - profile
5. dictionary to map profile statistic type to description
6. inputs (uplift, power, confidence level) for which random cohort sizes were estimated during an earlier step
7. type of audience strategy (single- or multi- group) from which cohorts were created during an earlier step

In [6]:
#| echo: true
# 1. 
best_run_id = "d5407876ef45489192743e6899207ca5"

# 2. categorical column
categorical_features = [
    "bounces",
    "last_action",
    "source",
    "medium",
    "channelGrouping",
    "browser",
    "os",
    "deviceCategory",
]

# 3. numerical columns
numerical_features = [
    "hits",
    "promos_displayed",
    "promos_clicked",
    "product_views",
    "product_clicks",
    "pageviews",
    "time_on_site",
]

# 4. GCP resources
gbq_dataset_id = 'mydemo2asdf'
gbq_table_id_cohorts = 'audience_cohorts'
gbq_table_id_profiles = 'audience_profiles'
gbq_table_feats_imps = 'audience_feats_imp'
gbq_table_id_summary = 'monthly_summary'
gbq_table_id_sa_fracs = "cohort_audience_fractions"
gbq_table_id_conv_rates = "audience_conversion_rates"
gbq_table_id_conv_rates_agg_combo = "conversion_rates_aggregated"
gbq_table_id_daily_perf = "daily_summary"
gbq_table_id_cat_feats_kpis = "categorical_features_kpis"

# 5. dictionary to map statistic type to description
stat_type_desc_mapper_dict = {
    "behavior": "Behavioral",
    "mean": "Descriptive statistics",
    "mode": "Descriptive statistics",
    "max": "Descriptive statistics",
    "feature_importance": "Most important ML features",
}

# 6. inputs used to estimate sample sizes
wanted_inputs = {
    "uplift_percentage": 10,
    "power_percentage": 55,
    "confidence_level_percentage": 55,
}

# 7. type of audience strategy to use when creating groups
audience_strategy = 1

::: {.content-hidden}
Get path to data sub-folders
:::

In [7]:
data_dir = os.path.join(PROJ_ROOT_DIR, "data")
raw_data_dir = os.path.join(data_dir, "raw")
processed_data_dir = os.path.join(data_dir, "processed")
gcp_keys_dir = os.path.join(PROJ_ROOT_DIR, "gcp_keys")

::: {.content-hidden}
Load Google Cloud authentication credentials for use with the native BigQuery Python client
:::

In [8]:
gcp_proj_id = os.environ["GCP_PROJECT_ID"]
gcp_creds_fpath = glob(os.path.join(gcp_keys_dir, "*", "*.json"))[0]
gcp_creds = service_account.Credentials.from_service_account_file(
    gcp_creds_fpath
)

::: {.content-hidden}
Get fully resolved name of the BigQuery table
:::

In [9]:
gbq_table_fully_resolved_cohorts = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_cohorts}"
gbq_table_fully_resolved_profiles = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_profiles}"
gbq_table_fully_resolved_feats_imp = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_feats_imps}"
gbq_summary_table_id_fully_resolved = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_summary}"
gbq_sa_fracs_table_id_fully_resolved = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_sa_fracs}"
gbq_conv_rates_table_id_fully_resolved = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_conv_rates}"
gbq_conv_rates_combo_table_id_fully_resolved = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_conv_rates_agg_combo}"
gbq_daily_perf_combo_table_id_fully_resolved = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_daily_perf}"
gbq_table_fully_resolved_cat_feat_kpis = f"{gcp_proj_id}.{gbq_dataset_id}.{gbq_table_id_cat_feats_kpis}"

::: {.content-hidden}
Create authenticated native BigQuery Python client
:::

In [10]:
client = bigquery.Client(project=gcp_proj_id, credentials=gcp_creds)

::: {.content-hidden} 
Get filepaths to audience

1. feature importances
2. profiles
:::

In [11]:
(
    fpath_feat_imps,
    fpath_profile,
    fpath_cohorts,
    fpath_development,
    fpath_sa_frac,
    fpath_conv_rates,
) = [
    glob(
        os.path.join(
            processed_data_dir, f"{prefix}_{best_run_id}__*.parquet.gzip"
        )
    )[-1]
    for prefix in [
        "audience_profiles_feature_importances__run",
        "audience_profiles__run",
        'audience_cohorts__run',
        "processed_data__run",
        "cohort_audience_fractions__run",
        "audience_development_inference_conversion_rates__run",
    ]
]
fpath_aud_sizes = glob(
    os.path.join(
        processed_data_dir, f"audience_sample_sizes__run_{best_run_id}.parquet.gzip"
    )
)[-1]

::: {.content-hidden}
Get inference data month from filepath
:::

In [12]:
#| output: false
infer_month_profile = fpath_profile.partition("infer_month_")[2].split("__")[0]
infer_month_feats = fpath_feat_imps.partition("infer_month_")[2].split("__")[0]
infer_month_cohort = fpath_cohorts.partition("infer_month_")[2].split("__")[0]
infer_month_frac = fpath_sa_frac.partition("infer_month_")[2].split("__")[0]
infer_month_conv_rates = fpath_conv_rates.partition("infer_month_")[2].split("__")[0]
try:
    assert infer_month_feats == infer_month_profile
    assert infer_month_cohort == infer_month_profile
    assert infer_month_frac == infer_month_profile
    assert infer_month_conv_rates == infer_month_profile
    print("Got same inference period from audience profile, feature importances and cohort")
except AssertionError as e:
    print(
        f"{str(e)}Did not get same inference period from audience profile and "
        "feature importance"
    )

Got same inference period from audience profile, feature importances and cohort


## Get Data

### Feature Importances

In [13]:
#| echo: true
df_feats = (
    pd.read_parquet(fpath_feat_imps)
    # select only wanted columns
    [['audience_strategy', 'num_observations', 'stat', 'maudience', 'value']]
)
df_feats.head()

,audience_strategy,num_observations,stat,maudience,value
0,1,500,medium__cpc,High,0.855569
1,1,500,os__FreeBSD,High,0.771953
2,1,500,browser__other,High,0.70914
3,1,500,os__Nokia,High,0.68788
4,1,500,hits,High,0.686577


### Audience Profiles

In [14]:
#| echo: true
df_profile = pd.read_parquet(fpath_profile)
df_profile.head()

,stat,stat_type,High,Low,Medium,feature,audience_strategy
0,hour__mean,mean,13.043304371810784,13.025513722245208,12.969241379310345,hour,1
1,day_of_week__mean,mean,4.009240104813129,3.9921390153082332,4.009931034482759,day_of_week,1
2,source__mode,mode,google,google,google,source,1
3,medium__mode,mode,organic,organic,organic,medium,1
4,channelGrouping__mode,mode,Organic Search,Organic Search,Organic Search,channelGrouping,1


### Audience Cohorts

In [15]:
#| echo: true
df_cohorts = pd.read_parquet(fpath_cohorts)

### Audience Sample (Cohort) Size Requirements and Metrics

In [16]:
#| echo: true
df_required_sample_sizes = (
    pd.read_parquet(
        fpath_aud_sizes,
        filters=[
            ('audience_strategy', '=', audience_strategy)
        ]# + [
        #     (k, "=", v)
        #     for k,v in zip(
        #         ['uplift', 'power', 'ci_level'],
        #         list(wanted_inputs.values()),
        #     )
        # ],
    )
    # select only wanted columns
    .drop(columns=['group_number', 'group_size_proportion'])
)

### ML Development Data

In [17]:
#| echo: true
df_development = pd.read_parquet(fpath_development)
df_development.head()

,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,...,browser,os,deviceCategory,added_to_cart,revenue,made_purchase_on_future_visit,split_type,score,predicted_score_label,predicted_label
0,9825248834764914907,1476553596,1,2016-10-15 10:46:36,4,10,15,7,10,46,...,Chrome,Chrome OS,desktop,0,<NA>,0,train_val,0.08047,True,0
1,5056863508248407677,1478647282,1,2016-11-08 15:21:22,4,11,8,3,15,21,...,Chrome,Linux,desktop,0,<NA>,0,train_val,0.169332,True,0
2,8458455338011744844,1480442988,1,2016-11-29 10:09:48,4,11,29,3,10,9,...,Chrome,Macintosh,desktop,1,<NA>,0,train_val,0.003096,True,0
3,1513972423387111118,1478028014,1,2016-11-01 12:20:14,4,11,1,3,12,20,...,Safari,iOS,mobile,0,<NA>,0,train_val,0.042589,True,0
4,3861070950942855450,1485365615,1,2017-01-25 09:33:35,1,1,25,4,9,33,...,Chrome,Windows,desktop,0,<NA>,0,train_val,0.253123,True,0


### Conversion Rates in Development and Inference

In [18]:
#| echo: true
df_conv_rates = pd.read_parquet(fpath_conv_rates)

### Estimated and Actual Cohort to Audience Fractions

In [19]:
#| echo: true
df_sa_frac = pd.read_parquet(fpath_sa_frac)

## Transform Data

### Feature Importances

In [20]:
#| echo: true
df_feats = (
    df_feats.assign(
        stat=lambda df: (
            df["stat"].str.replace("__", " = ")
            .str.replace(" = 1", " = True")
            .str.replace("_", " ")
        )
    )
)
display(df_feats.head())
display(df_feats.tail())

,audience_strategy,num_observations,stat,maudience,value
0,1,500,medium = cpc,High,0.855569
1,1,500,os = FreeBSD,High,0.771953
2,1,500,browser = other,High,0.70914
3,1,500,os = Nokia,High,0.68788
4,1,500,hits,High,0.686577


,audience_strategy,num_observations,stat,maudience,value
25,1,500,bounces = True,Low,0.708115
26,1,500,os = BlackBerry,Low,0.572242
27,1,500,medium = (not set),Low,0.433122
28,1,500,os = Nokia,Low,0.200427
29,1,500,source = google,Low,0.144921


### Audience Profile

In [21]:
#| echo: true
df_profile_sliced = (
    df_profile
    # get behavioral and (average) descriptive statistics
    .query("(stat_type == 'behavior') | (stat.str.endswith('mean'))")
    # make stat column more reader friendly
    .assign(
        stat_expanded=lambda df: (
            df["stat"]
            .str.replace("__1", " = True")
            .str.replace("__0", " = False")
            .str.replace("__", " = ")
            .str.replace("_", " ")
            .str.replace("= mean", "(Mean)")
            .str.title()
        )
    )
    # drop unwanted columns
    .drop(columns=['stat'])
    .reset_index()
    # rename columns to titlecase
    .rename(columns=str.title)
    # set datatypes
    .astype(
        {
            'Audience_Strategy': pd.Int8Dtype(),
            "Stat_Expanded": pd.StringDtype(),
            "High": pd.Float32Dtype(),
            "Medium": pd.Float32Dtype(),
            "Low": pd.Float32Dtype(),
        }
    )
    # select only wanted columns
    [['Audience_Strategy', 'Stat_Expanded', 'High', 'Medium', 'Low']]
)
df_profile_sliced

,Audience_Strategy,Stat_Expanded,High,Medium,Low
0,1,Hour (Mean),13.043304,12.969241,13.025514
1,1,Day Of Week (Mean),4.00924,4.009931,3.992139
2,1,Hits (Mean),6.214315,6.356552,6.497587
3,1,Promos Displayed (Mean),8.550683,8.541931,8.612743
4,1,Promos Clicked (Mean),0.0,0.0,0.000138
5,1,Product Views (Mean),22.395256,23.241379,23.932423
6,1,Product Clicks (Mean),0.647083,0.679448,0.704455
7,1,Pageviews (Mean),5.287409,5.405241,5.49207
8,1,Revenue (Mean),179.237625,178.182846,193.435837
9,1,Added To Cart (Mean),0.19487,0.201241,0.19418


### Audience Cohorts

In [22]:
#| echo: true
df_cohorts = (
    df_cohorts
    .assign(made_purchase_on_future_visit=None)
    .assign(split_type='infer')
    .assign(infer_month=infer_month_feats)
    .astype(
        {
            'split_type': pd.StringDtype(),
            "infer_month": pd.StringDtype(),
            "made_purchase_on_future_visit": pd.BooleanDtype(),
        }
    )
)
col = df_cohorts.pop("infer_month")
df_cohorts.insert(0, col.name, col)
with pd.option_context('display.max_columns', None):
    display(df_cohorts.head())
    display(df_cohorts.tail())

,infer_month,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,revenue,score,predicted_score_label,maudience,cohort,audience_strategy,made_purchase_on_future_visit,split_type
0,March,9725965720822289910,1490389872,1,2017-03-24 14:11:12,1,3,24,6,14,11,12,mall.googleplex.com,referral,Referral,4,0,Unknown,18,0,12,0,4,66,Chrome,Linux,desktop,0,<NA>,0.960677,False,High,Control,1,<NA>,infer
1,March,1262291561527945490,1490385910,1,2017-03-24 13:05:10,1,3,24,6,13,5,10,(direct),(none),Direct,5,0,Unknown,18,0,25,0,5,99,Chrome,iOS,mobile,0,<NA>,0.879824,False,High,Control,1,<NA>,infer
2,March,9189040746503498111,1488898168,1,2017-03-07 06:49:28,1,3,7,3,6,49,28,(direct),(none),Direct,1,1,Unknown,0,0,0,0,1,0,Chrome,Android,mobile,0,<NA>,0.867118,False,High,Control,1,<NA>,infer
3,March,1844223110946928452,1490399038,1,2017-03-24 16:43:58,1,3,24,6,16,43,58,google,organic,Organic Search,20,0,Add product(s) to cart,9,0,90,3,15,414,Chrome,Macintosh,desktop,2,<NA>,0.864372,True,High,Control,1,<NA>,infer
4,March,5443907013787165361,1490637573,1,2017-03-27 10:59:33,1,3,27,2,10,59,33,google,organic,Organic Search,7,0,Unknown,9,0,47,0,7,41,Chrome,Macintosh,desktop,0,<NA>,0.860614,False,High,Control,1,<NA>,infer


,infer_month,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,revenue,score,predicted_score_label,maudience,cohort,audience_strategy,made_purchase_on_future_visit,split_type
21747,March,5476993230782741357,1490986341,1,2017-03-31 11:52:21,1,3,31,6,11,52,21,(direct),(none),Direct,18,0,Add product(s) to cart,0,0,107,3,14,551,Chrome,Linux,desktop,1,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21748,March,6629693276913663746,1488520182,1,2017-03-02 21:49:42,1,3,2,5,21,49,42,google,organic,Organic Search,3,0,Unknown,9,0,12,0,3,19,Chrome,Macintosh,desktop,0,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21749,March,7820592629354691307,1489762686,1,2017-03-17 07:58:06,1,3,17,6,7,58,6,google,organic,Organic Search,2,0,Unknown,9,0,0,0,2,8,Chrome,Windows,desktop,0,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21750,March,9113372584810915471,1489627383,1,2017-03-15 18:23:03,1,3,15,4,18,23,3,youtube.com,referral,Social,6,0,Unknown,36,0,24,0,6,880,Chrome,Android,mobile,0,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21751,March,8899670647171745453,1488901570,1,2017-03-07 07:46:10,1,3,7,3,7,46,10,google,organic,Organic Search,1,1,Unknown,0,0,12,0,1,0,Chrome,Windows,desktop,0,<NA>,0.0,True,Low,<NA>,1,<NA>,infer


### Conversion Rates in Development and Inference

::: {.content-hidden}
Insert inference month and audience strategy columns
:::

In [23]:
#| echo: true
df_conv_rates = (
    df_conv_rates
    .assign(infer_month=[None]*3 + [infer_month_conv_rates]*3)
    .assign(audience_strategy=audience_strategy)
    .astype(
        {
            "infer_month": pd.StringDtype(),
            "audience_strategy": pd.Int8Dtype(),
        }
    )
    .assign(
        data_type=lambda df: df['data_type'].str.replace('_', ' ').str.title()
    )
    .fillna(
        {
            "true_conv_rate": 0,
            "overall_true_conv_rate": 0,
            "true_conversions": 0,
            "pred_conv_rate": 0,
            "total_visitors": 0,
            'min_score': 0,
        }
    )
)
col = df_conv_rates.pop("infer_month")
df_conv_rates.insert(0, col.name, col)
col = df_conv_rates.pop("audience_strategy")
df_conv_rates.insert(0, col.name, col)
df_conv_rates

,audience_strategy,infer_month,maudience,pred_conversions,total_visitors,min_score,true_conversions,data_type,data_size,true_conv_rate,overall_true_conv_rate,pred_conv_rate,overall_pred_conv_rate
0,1,<NA>,High,1125,6722,0.140977,165,Development,20164,2.454627,2.30609,16.73609,5.57925
1,1,<NA>,Low,0,6721,0.0,147,Development,20164,2.187175,2.30609,0.0,5.57925
2,1,<NA>,Medium,0,6721,0.022545,153,Development,20164,2.276447,2.30609,0.0,5.57925
3,1,March,High,1243,7251,0.143627,0,Inference,21752,0.0,0.0,17.142464,5.714417
4,1,March,Low,0,7251,0.0,0,Inference,21752,0.0,0.0,0.0,5.714417
5,1,March,Medium,0,7250,0.022746,0,Inference,21752,0.0,0.0,0.0,5.714417


### Conversion Rates Aggregated by Audience Group

In [24]:
#| echo: true
df_conv_rates_by_aud = (
    df_conv_rates[['maudience', 'data_type', 'true_conv_rate', 'pred_conv_rate']].melt(
        id_vars=['maudience', 'data_type'],
        value_vars=['true_conv_rate', 'pred_conv_rate'],
        value_name='value',
        var_name='var'
    )
    .assign(var=lambda df: df['var'].str.replace("_conv_rate", ""))
    .astype(
        {
            "maudience": pd.StringDtype(),
            'data_type': pd.StringDtype(),
            "var": pd.StringDtype(),
            "value": pd.Float32Dtype(),
        }
    )
)
df_conv_rates_by_aud

,maudience,data_type,var,value
0,High,Development,true,2.454627
1,Low,Development,true,2.187175
2,Medium,Development,true,2.276447
3,High,Inference,true,0.0
4,Low,Inference,true,0.0
5,Medium,Inference,true,0.0
6,High,Development,pred,16.73609
7,Low,Development,pred,0.0
8,Medium,Development,pred,0.0
9,High,Inference,pred,17.142464


### Conversion Rates Aggregated Overall

In [25]:
#| echo: true
df_conv_rates_overall = (
    df_conv_rates[['data_type', 'overall_true_conv_rate', 'overall_pred_conv_rate', 'data_size']]
    .groupby(['data_type'], as_index=False).mean()
    .melt(
        id_vars=['data_type'],
        value_vars=['overall_pred_conv_rate', 'overall_true_conv_rate', 'data_size'],
        value_name='value',
        var_name='var'
    )
    .assign(var=lambda df: df['var'].str.replace("_", " ").str.title())
    .astype(
        {
            "data_type": pd.StringDtype(),
            "var": pd.StringDtype(),
            "value": pd.Float32Dtype(),
        }
    )
    .assign(maudience=None)
    .astype({"maudience": pd.StringDtype()})
)
df_conv_rates_overall

,data_type,var,value,maudience
0,Development,Overall Pred Conv Rate,5.57925,<NA>
1,Inference,Overall Pred Conv Rate,5.714417,<NA>
2,Development,Overall True Conv Rate,2.30609,<NA>
3,Inference,Overall True Conv Rate,0.0,<NA>
4,Development,Data Size,20164.0,<NA>
5,Inference,Data Size,21752.0,<NA>


### Combine Aggregated Conversion Rates

::: {.content-hidden}
Verify that columns in overall and per-audience conversion rates `DataFrame`s are identical
:::

In [26]:
assert df_conv_rates_by_aud.shape[1] == df_conv_rates_overall.shape[1]
assert list(df_conv_rates_overall[list(df_conv_rates_by_aud)]) == list(df_conv_rates_by_aud)

::: {.content-hidden}
Combine overall and per-audience conversion rates data
:::

In [27]:
#| echo: true
df_conv_rates_agg_combo = pd.concat([df_conv_rates_by_aud, df_conv_rates_overall], ignore_index=True)
df_conv_rates_agg_combo

,maudience,data_type,var,value
0,High,Development,true,2.454627
1,Low,Development,true,2.187175
2,Medium,Development,true,2.276447
3,High,Inference,true,0.0
4,Low,Inference,true,0.0
5,Medium,Inference,true,0.0
6,High,Development,pred,16.73609
7,Low,Development,pred,0.0
8,Medium,Development,pred,0.0
9,High,Inference,pred,17.142464


### Estimated and Actual Cohort to Audience Fractions

::: {.content-hidden}
Insert inference month and audience strategy columns
:::

In [28]:
#| echo: true
df_sa_frac = (
    df_sa_frac
    .assign(infer_month=[None]*6 + [infer_month_frac]*6)
    .assign(audience_strategy=audience_strategy)
    .astype(
        {
            "infer_month": pd.StringDtype(),
            "audience_strategy": pd.Int8Dtype(),
        }
    )
)
col = df_sa_frac.pop("infer_month")
df_sa_frac.insert(0, col.name, col)
col = df_sa_frac.pop("audience_strategy")
df_sa_frac.insert(0, col.name, col)
df_sa_frac

,audience_strategy,infer_month,maudience,cohort,size,group_size,uplift,power,ci_level,samp_to_aud_frac,size_type,data_type,data_size
0,1,<NA>,High,Test,1933,6721,10,55,55,28.760601,required,development,20164
1,1,<NA>,High,Control,1933,6721,10,55,55,28.760601,required,development,20164
2,1,<NA>,Medium,Test,2088,6721,10,55,55,31.066805,required,development,20164
3,1,<NA>,Medium,Control,2088,6721,10,55,55,31.066805,required,development,20164
4,1,<NA>,Low,Test,2176,6722,10,55,55,32.371319,required,development,20164
5,1,<NA>,Low,Control,2176,6722,10,55,55,32.371319,required,development,20164
6,1,March,High,Control,2085,7251,10,55,55,28.754654,randomly selected,inference,21752
7,1,March,High,Test,2085,7251,10,55,55,28.754654,randomly selected,inference,21752
8,1,March,Low,Control,2347,7251,10,55,55,32.36795,randomly selected,inference,21752
9,1,March,Low,Test,2347,7251,10,55,55,32.36795,randomly selected,inference,21752


::: {.callout-note title="Notes"}

1. The accuracy of the predicted propensities are different between the `inference` and `test_split` datasets. The model performance is relatively worse on the unseen data (`inference`) compared to the data seen during development (`test_split`). Audience groups (bins) are created based on these predicted propensities. So, this ML model inaccuracy can create bins with different sizes between these two datasets. Additionally, if the number of visitors in the `inference` and `test_split` datasets are different, then this also contributes to differently sized bins. Both of these factors (inaccuracy and differently sized datasets - see the `data_size` column) are present here and this explains why the following columns
   - predicted conversion rate (`pred_conv_rate`)
   - cohort size (`size`)
   - cohort-to-audience fraction (`sample_to_audience_frac`)
   
   are different between the test split (during ML model development) and inference (during production).
3. Similarly, the predicted (`pred_conv_rate`) and true (`true_conv_rate`) conversion rates are for the test split due to the poor accuracy of the ML model's predictions.
:::

### ML Development Data

In [29]:
#| echo: true
df_development = (
    df_development
    .assign(infer_month=None)
    .drop(columns=['predicted_label'])
    .assign(maudience=None)
    .assign(cohort=None)
    .assign(audience_strategy=None)
    .astype(
        {
            "infer_month": pd.StringDtype(),
            "maudience": pd.StringDtype(),
            "cohort": pd.StringDtype(),
            "audience_strategy": pd.Int8Dtype(),
        }
    )
)
col = df_development.pop("infer_month")
df_development.insert(0, col.name, col)
with pd.option_context('display.max_columns', None):
    display(df_development.head())
    display(df_development.tail())

,infer_month,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,revenue,made_purchase_on_future_visit,split_type,score,predicted_score_label,maudience,cohort,audience_strategy
0,<NA>,9825248834764914907,1476553596,1,2016-10-15 10:46:36,4,10,15,7,10,46,36,google,organic,Organic Search,8,0,Unknown,9,0,70,0,8,102,Chrome,Chrome OS,desktop,0,<NA>,0,train_val,0.08047,True,<NA>,<NA>,<NA>
1,<NA>,5056863508248407677,1478647282,1,2016-11-08 15:21:22,4,11,8,3,15,21,22,mall.googleplex.com,referral,Referral,7,0,Unknown,18,0,42,0,7,46,Chrome,Linux,desktop,0,<NA>,0,train_val,0.169332,True,<NA>,<NA>,<NA>
2,<NA>,8458455338011744844,1480442988,1,2016-11-29 10:09:48,4,11,29,3,10,9,48,mall.googleplex.com,referral,Referral,24,0,Remove product(s) from cart,18,2,62,5,15,1578,Chrome,Macintosh,desktop,1,<NA>,0,train_val,0.003096,True,<NA>,<NA>,<NA>
3,<NA>,1513972423387111118,1478028014,1,2016-11-01 12:20:14,4,11,1,3,12,20,14,(direct),(none),Direct,1,1,Unknown,0,0,0,0,1,0,Safari,iOS,mobile,0,<NA>,0,train_val,0.042589,True,<NA>,<NA>,<NA>
4,<NA>,3861070950942855450,1485365615,1,2017-01-25 09:33:35,1,1,25,4,9,33,35,groups.google.com,referral,Social,3,0,Unknown,9,0,11,0,3,21,Chrome,Windows,desktop,0,<NA>,0,train_val,0.253123,True,<NA>,<NA>,<NA>


,infer_month,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,revenue,made_purchase_on_future_visit,split_type,score,predicted_score_label,maudience,cohort,audience_strategy
133887,<NA>,2859155514259411479,1486248414,1,2017-02-04 14:46:54,1,2,4,7,14,46,54,google,organic,Organic Search,3,0,Unknown,0,0,17,0,3,40,Chrome,Android,mobile,0,<NA>,0,test,0.495976,True,<NA>,<NA>,<NA>
133888,<NA>,460482791086125299,1486859939,1,2017-02-11 16:38:59,1,2,11,7,16,38,59,google,organic,Organic Search,3,0,Unknown,0,0,12,0,3,50,Safari,iOS,mobile,0,<NA>,0,test,0.070427,True,<NA>,<NA>,<NA>
133889,<NA>,1197241994723568470,1487107604,1,2017-02-14 13:26:44,1,2,14,3,13,26,44,(direct),(none),Direct,3,0,Unknown,0,0,17,0,3,15,Chrome,Windows,desktop,0,<NA>,0,test,0.169552,True,<NA>,<NA>,<NA>
133890,<NA>,91736817749198966,1488142038,1,2017-02-26 12:47:18,1,2,26,1,12,47,18,google,organic,Organic Search,3,0,Unknown,0,0,27,0,3,76,Chrome,Chrome OS,desktop,0,<NA>,0,test,0.096793,True,<NA>,<NA>,<NA>
133891,<NA>,4990910589525036851,1486257216,1,2017-02-04 17:13:36,1,2,4,7,17,13,36,youtube.com,referral,Social,3,0,Unknown,0,0,30,0,3,91,Edge,Windows,desktop,0,<NA>,0,test,0.00032,True,<NA>,<NA>,<NA>


### Combine Development and Cohorts Data

::: {.content-hidden}
Verify that columns in cohorts and development `DataFrame`s are identical
:::

In [30]:
assert df_cohorts.shape[1] == df_development.shape[1]
assert list(df_development[list(df_cohorts)]) == list(df_cohorts)

::: {.content-hidden}
Combine development and cohorts data
:::

In [31]:
#| echo: true
df_dev_cohorts = (
    pd.concat([df_development[list(df_cohorts)], df_cohorts])
    .astype(
        {
            "source": pd.CategoricalDtype(),
            "medium": pd.CategoricalDtype(),
            "channelGrouping": pd.CategoricalDtype(),
            "browser": pd.CategoricalDtype(),
            "os": pd.CategoricalDtype(),
            "made_purchase_on_future_visit": pd.BooleanDtype(),
            "split_type": pd.StringDtype(),
        }
    )
)
with pd.option_context('display.max_columns', None):
    display(df_dev_cohorts.head())
    display(df_dev_cohorts.tail())

,infer_month,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,revenue,score,predicted_score_label,maudience,cohort,audience_strategy,made_purchase_on_future_visit,split_type
0,<NA>,9825248834764914907,1476553596,1,2016-10-15 10:46:36,4,10,15,7,10,46,36,google,organic,Organic Search,8,0,Unknown,9,0,70,0,8,102,Chrome,Chrome OS,desktop,0,<NA>,0.08047,True,<NA>,<NA>,<NA>,False,train_val
1,<NA>,5056863508248407677,1478647282,1,2016-11-08 15:21:22,4,11,8,3,15,21,22,mall.googleplex.com,referral,Referral,7,0,Unknown,18,0,42,0,7,46,Chrome,Linux,desktop,0,<NA>,0.169332,True,<NA>,<NA>,<NA>,False,train_val
2,<NA>,8458455338011744844,1480442988,1,2016-11-29 10:09:48,4,11,29,3,10,9,48,mall.googleplex.com,referral,Referral,24,0,Remove product(s) from cart,18,2,62,5,15,1578,Chrome,Macintosh,desktop,1,<NA>,0.003096,True,<NA>,<NA>,<NA>,False,train_val
3,<NA>,1513972423387111118,1478028014,1,2016-11-01 12:20:14,4,11,1,3,12,20,14,(direct),(none),Direct,1,1,Unknown,0,0,0,0,1,0,Safari,iOS,mobile,0,<NA>,0.042589,True,<NA>,<NA>,<NA>,False,train_val
4,<NA>,3861070950942855450,1485365615,1,2017-01-25 09:33:35,1,1,25,4,9,33,35,groups.google.com,referral,Social,3,0,Unknown,9,0,11,0,3,21,Chrome,Windows,desktop,0,<NA>,0.253123,True,<NA>,<NA>,<NA>,False,train_val


,infer_month,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,revenue,score,predicted_score_label,maudience,cohort,audience_strategy,made_purchase_on_future_visit,split_type
21747,March,5476993230782741357,1490986341,1,2017-03-31 11:52:21,1,3,31,6,11,52,21,(direct),(none),Direct,18,0,Add product(s) to cart,0,0,107,3,14,551,Chrome,Linux,desktop,1,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21748,March,6629693276913663746,1488520182,1,2017-03-02 21:49:42,1,3,2,5,21,49,42,google,organic,Organic Search,3,0,Unknown,9,0,12,0,3,19,Chrome,Macintosh,desktop,0,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21749,March,7820592629354691307,1489762686,1,2017-03-17 07:58:06,1,3,17,6,7,58,6,google,organic,Organic Search,2,0,Unknown,9,0,0,0,2,8,Chrome,Windows,desktop,0,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21750,March,9113372584810915471,1489627383,1,2017-03-15 18:23:03,1,3,15,4,18,23,3,youtube.com,referral,Social,6,0,Unknown,36,0,24,0,6,880,Chrome,Android,mobile,0,<NA>,0.0,False,Low,<NA>,1,<NA>,infer
21751,March,8899670647171745453,1488901570,1,2017-03-07 07:46:10,1,3,7,3,7,46,10,google,organic,Organic Search,1,1,Unknown,0,0,12,0,1,0,Chrome,Windows,desktop,0,<NA>,0.0,True,Low,<NA>,1,<NA>,infer


### Daily Performance Summary in Combined Data, by Audience Group

Get daily metadata by audience group

In [32]:
#| echo: true
df_aud_hmap = (
    trh.perform_custom_aggregation(
        (
            df_dev_cohorts
            .sort_values(by=['visitStartTime'], ignore_index=True)
            .assign(date=lambda df: pd.to_datetime(df['visitStartTime']).dt.date)
            .assign(maudience=lambda df: df['maudience'].fillna("Development"))
        ),
        groupby_cols=['month', 'date', 'maudience'],
        agg_dict={
            "made_purchase_on_future_visit": ["sum"],
            "revenue": "sum",
            "fullvisitorid": "count",
            "added_to_cart": "sum",
            "pageviews": "sum",
            "time_on_site": "mean",
            "product_views": "sum",
            "product_clicks": "sum",
            "bounces": "sum",
        },
        audience_strategy=df_cohorts['audience_strategy'].unique().tolist()[0],
        column_renamer={
            "made_purchase_on_future_visit_sum": "return_purchasers",
            "revenue_sum": "revenue",
            "added_to_cart_sum": "add_to_cart",
            "fullvisitorid_count": "visitors",
            "product_views_sum": "product_views",
            "product_clicks_sum": "product_clicks",
            "pageviews_sum": "pageviews",
            "bounces_sum": "bounces",
            "time_on_site_mean": "time_on_site",
        },
        visitor_type_mapper=dict(
            zip(
                ['Development', 'High', 'Medium', 'Low'],
                ['return_purchasers', 'all_visitors', 'all_visitors', 'all_visitors']
            )
        ),
        dtypes_out={
            "month": pd.StringDtype(),
            "date": pd.StringDtype(),
            "maudience": pd.StringDtype(),
            "return_purchasers": pd.Int16Dtype(),
            "revenue": pd.Float32Dtype(),
            "visitors": pd.Int16Dtype(),
            "add_to_cart": pd.Int32Dtype(),
            "pageviews": pd.Int32Dtype(),
            "time_on_site": pd.Float32Dtype(),
            "product_views": pd.Int32Dtype(),
            "product_clicks": pd.Int32Dtype(),
            "bounces": pd.Int32Dtype(),
            'audience_strategy': pd.Int8Dtype(),
            "bounce_rate": pd.Float32Dtype(),
            "product_clicks_rate": pd.Float32Dtype(),
            "add_to_cart_rate": pd.Float32Dtype(),
            "visitor_type": pd.StringDtype(),
        },
    )
    .assign(agg_type='audience_group')
    .astype({"agg_type": pd.StringDtype()})
)
display(df_aud_hmap.head())
display(df_aud_hmap.tail())

,month,date,maudience,return_purchasers,revenue,visitors,add_to_cart,pageviews,time_on_site,product_views,product_clicks,bounces,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,agg_type
0,September,2016-09-01,Development,21,824.710022,718,180,4894,3.087837,53001,892,202,1,28.133705,1.682987,25.069637,return_purchasers,audience_group
1,September,2016-09-02,Development,16,1087.579956,636,125,4311,3.174974,49580,816,193,1,30.345911,1.645825,19.654089,return_purchasers,audience_group
2,September,2016-09-03,Development,3,514.880005,387,80,2355,2.39186,25808,466,124,1,32.041344,1.805642,20.671835,return_purchasers,audience_group
3,September,2016-09-04,Development,7,163.350006,349,66,2136,2.962273,25532,353,114,1,32.664757,1.382579,18.911175,return_purchasers,audience_group
4,September,2016-09-05,Development,11,483.380005,469,89,3034,3.292111,36007,563,167,1,35.607677,1.563585,18.976545,return_purchasers,audience_group


,month,date,maudience,return_purchasers,revenue,visitors,add_to_cart,pageviews,time_on_site,product_views,product_clicks,bounces,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,agg_type
269,March,2017-03-30,High,0,2129.51001,336,56,1619,2.871478,6749,204,122,1,36.309525,3.02267,16.666666,all_visitors,audience_group
270,March,2017-03-30,Medium,0,0.0,331,40,1366,2.371299,6053,209,149,1,45.015106,3.452833,12.084592,all_visitors,audience_group
271,March,2017-03-31,Low,0,891.940002,270,40,1337,3.308272,5599,165,100,1,37.037037,2.946955,14.814815,all_visitors,audience_group
272,March,2017-03-31,High,0,1045.080078,267,47,1349,2.470787,5348,170,87,1,32.58427,3.178758,17.602997,all_visitors,audience_group
273,March,2017-03-31,Medium,0,2137.109863,262,37,1397,3.320611,6279,159,85,1,32.442749,2.53225,14.122137,all_visitors,audience_group


::: {.content-hidden}
Verify that there are no duplicated dates, since the stats were aggregated daily and by audience group
:::

In [33]:
assert df_aud_hmap[df_aud_hmap.duplicated(subset=['maudience', 'date'], keep=False)].empty

### Daily Performance Summary in Overall Combined Data

Get daily metadata overall

In [34]:
#| echo: false
#| output: true
df_hmap = (
    trh.perform_custom_aggregation(
        (
            df_dev_cohorts
            .sort_values(by=['visitStartTime'], ignore_index=True)
            .assign(date=lambda df: pd.to_datetime(df['visitStartTime']).dt.date)
            .assign(maudience=lambda df: df['maudience'].fillna("Development"))
        ),
        groupby_cols=['month', 'date'],
        agg_dict={
            "made_purchase_on_future_visit": ["sum"],
            "revenue": "sum",
            "fullvisitorid": "count",
            "added_to_cart": "sum",
            "pageviews": "sum",
            "time_on_site": "mean",
            "product_views": "sum",
            "product_clicks": "sum",
            "bounces": "sum",
        },
        audience_strategy=df_cohorts['audience_strategy'].unique().tolist()[0],
        column_renamer={
            "made_purchase_on_future_visit_sum": "return_purchasers",
            "revenue_sum": "revenue",
            "added_to_cart_sum": "add_to_cart",
            "fullvisitorid_count": "visitors",
            "product_views_sum": "product_views",
            "product_clicks_sum": "product_clicks",
            "pageviews_sum": "pageviews",
            "bounces_sum": "bounces",
            "time_on_site_mean": "time_on_site",
        },
        visitor_type_mapper=dict(
            zip(
                [
                    'September',
                    'October',
                    'November',
                    'December',
                    'January',
                    'February',
                    'March',
                ],
                [
                    'return_purchasers',
                    'return_purchasers',
                    'return_purchasers',
                    'return_purchasers',
                    'return_purchasers',
                    'return_purchasers',
                    'all_visitors',
                ],
            )
        ),
        visitor_type_col='month',
        dtypes_out={
            "month": pd.StringDtype(),
            "date": pd.StringDtype(),
            "return_purchasers": pd.Int16Dtype(),
            "revenue": pd.Float32Dtype(),
            "visitors": pd.Int16Dtype(),
            "add_to_cart": pd.Int32Dtype(),
            "pageviews": pd.Int32Dtype(),
            "time_on_site": pd.Float32Dtype(),
            "product_views": pd.Int32Dtype(),
            "product_clicks": pd.Int32Dtype(),
            "bounces": pd.Int32Dtype(),
            'audience_strategy': pd.Int8Dtype(),
            "bounce_rate": pd.Float32Dtype(),
            "product_clicks_rate": pd.Float32Dtype(),
            "add_to_cart_rate": pd.Float32Dtype(),
            "visitor_type": pd.StringDtype(),
        },
    )
    .assign(agg_type='overall')
    .assign(
        maudience=lambda df: df['month'].map(
            dict(
                zip(
                    [
                        'September',
                        'October',
                        'November',
                        'December',
                        'January',
                        'February',
                        'March',
                    ],
                    [
                        'Development',
                        'Development',
                        'Development',
                        'Development',
                        'Development',
                        'Development',
                        'Inference',
                    ],
                )
            )
        )
        
    )
    .astype({"agg_type": pd.StringDtype(), 'maudience': pd.StringDtype()})
)
display(df_hmap.head())
display(df_hmap.tail())

,month,date,return_purchasers,revenue,visitors,add_to_cart,pageviews,time_on_site,product_views,product_clicks,bounces,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,agg_type,maudience
0,September,2016-09-01,21,824.710022,718,180,4894,3.087837,53001,892,202,1,28.133705,1.682987,25.069637,return_purchasers,overall,Development
1,September,2016-09-02,16,1087.579956,636,125,4311,3.174974,49580,816,193,1,30.345911,1.645825,19.654089,return_purchasers,overall,Development
2,September,2016-09-03,3,514.880005,387,80,2355,2.39186,25808,466,124,1,32.041344,1.805642,20.671835,return_purchasers,overall,Development
3,September,2016-09-04,7,163.350006,349,66,2136,2.962273,25532,353,114,1,32.664757,1.382579,18.911175,return_purchasers,overall,Development
4,September,2016-09-05,11,483.380005,469,89,3034,3.292111,36007,563,167,1,35.607677,1.563585,18.976545,return_purchasers,overall,Development


,month,date,return_purchasers,revenue,visitors,add_to_cart,pageviews,time_on_site,product_views,product_clicks,bounces,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,agg_type,maudience
207,March,2017-03-27,0,7658.660156,823,182,4468,3.279364,18306,566,241,1,29.28311,3.091882,22.114216,all_visitors,overall,Inference
208,March,2017-03-28,0,6920.870117,854,114,4192,3.085851,17574,527,315,1,36.885246,2.998748,13.348947,all_visitors,overall,Inference
209,March,2017-03-29,0,5109.200195,839,168,4579,3.238836,18880,683,274,1,32.657925,3.617585,20.023838,all_visitors,overall,Inference
210,March,2017-03-30,0,3357.469971,976,166,4701,2.78943,20800,613,390,1,39.959015,2.947115,17.008196,all_visitors,overall,Inference
211,March,2017-03-31,0,4074.129883,799,124,4083,3.032457,17226,494,272,1,34.042553,2.867758,15.5194,all_visitors,overall,Inference


::: {.content-hidden}
Verify that there are no duplicated dates, since the stats were aggregated daily
:::

In [35]:
assert df_hmap[df_hmap.duplicated(subset=['date'], keep=False)].empty

### Combined Daily Performance Summary

::: {.content-hidden}
Verify that columns in overall daily summaries and by audience group `DataFrame`s are identical
:::

In [36]:
assert df_hmap.shape[1] == df_aud_hmap.shape[1]
assert list(df_hmap[list(df_aud_hmap)]) == list(df_aud_hmap)

::: {.content-hidden}
Combine daily performance summary data
:::

In [37]:
#| echo: true
df_hmap_combo = pd.concat([df_aud_hmap, df_hmap], ignore_index=True)
display(df_hmap_combo.head())
display(df_hmap_combo.tail())

,month,date,maudience,return_purchasers,revenue,visitors,add_to_cart,pageviews,time_on_site,product_views,product_clicks,bounces,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,agg_type
0,September,2016-09-01,Development,21,824.710022,718,180,4894,3.087837,53001,892,202,1,28.133705,1.682987,25.069637,return_purchasers,audience_group
1,September,2016-09-02,Development,16,1087.579956,636,125,4311,3.174974,49580,816,193,1,30.345911,1.645825,19.654089,return_purchasers,audience_group
2,September,2016-09-03,Development,3,514.880005,387,80,2355,2.39186,25808,466,124,1,32.041344,1.805642,20.671835,return_purchasers,audience_group
3,September,2016-09-04,Development,7,163.350006,349,66,2136,2.962273,25532,353,114,1,32.664757,1.382579,18.911175,return_purchasers,audience_group
4,September,2016-09-05,Development,11,483.380005,469,89,3034,3.292111,36007,563,167,1,35.607677,1.563585,18.976545,return_purchasers,audience_group


,month,date,maudience,return_purchasers,revenue,visitors,add_to_cart,pageviews,time_on_site,product_views,product_clicks,bounces,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,agg_type
481,March,2017-03-27,Inference,0,7658.660156,823,182,4468,3.279364,18306,566,241,1,29.28311,3.091882,22.114216,all_visitors,overall
482,March,2017-03-28,Inference,0,6920.870117,854,114,4192,3.085851,17574,527,315,1,36.885246,2.998748,13.348947,all_visitors,overall
483,March,2017-03-29,Inference,0,5109.200195,839,168,4579,3.238836,18880,683,274,1,32.657925,3.617585,20.023838,all_visitors,overall
484,March,2017-03-30,Inference,0,3357.469971,976,166,4701,2.78943,20800,613,390,1,39.959015,2.947115,17.008196,all_visitors,overall
485,March,2017-03-31,Inference,0,4074.129883,799,124,4083,3.032457,17226,494,272,1,34.042553,2.867758,15.5194,all_visitors,overall


### Monthly Performance Summary in Combined Data

Get aggregated summary of metadata by month

In [38]:
#| echo: false
#| output: true
df_summary = trh.perform_custom_aggregation(
    df_dev_cohorts,
    groupby_cols=['split_type', 'month'],
    agg_dict={
        "made_purchase_on_future_visit": ["sum"],
        "revenue": "sum",
        "fullvisitorid": "count",
        "added_to_cart": "sum",
        "pageviews": "sum",
        "time_on_site": "mean",
        "channelGrouping": pd.Series.mode,
        "deviceCategory": pd.Series.mode,
        "browser": pd.Series.mode,
        "os": pd.Series.mode,
        "product_views": "sum",
        "product_clicks": "sum",
        "bounces": "sum",
    },
    audience_strategy=df_cohorts['audience_strategy'].unique().tolist()[0],
    column_renamer={
        "made_purchase_on_future_visit_sum": "return_purchasers",
        "revenue_sum": "revenue",
        "added_to_cart_sum": "add_to_cart",
        "fullvisitorid_count": "visitors",
        "channelGrouping_mode": "channelGrouping",
        "deviceCategory_mode": "deviceCategory",
        "browser_mode": "browser",
        "os_mode": "os",
        "product_views_sum": "product_views",
        "product_clicks_sum": "product_clicks",
        "pageviews_sum": "pageviews",
        "bounces_sum": "bounces",
        "time_on_site_mean": "time_on_site",
    },
    visitor_type_mapper=dict(
        zip(
            ['train_val', 'test', 'infer'],
            ['return_purchasers', 'return_purchasers', 'all_visitors']
        )
    ),
    dtypes_out={
        'audience_strategy': pd.Int8Dtype(),
        "revenue": pd.Float32Dtype(),
        "visitors": pd.Int16Dtype(),
        "return_purchasers": pd.Int16Dtype(),
        "conversion_rate": pd.Float32Dtype(),
        "add_to_cart_rate": pd.Float32Dtype(),
        "product_clicks_rate": pd.Float32Dtype(),
        "pageviews": pd.Int32Dtype(),
        "bounce_rate": pd.Float32Dtype(),
        "time_on_site": pd.Float32Dtype(),
        "channelGrouping": pd.StringDtype(),
        "deviceCategory": pd.StringDtype(),
        "browser": pd.StringDtype(),
        "os": pd.StringDtype(),
        "split_type": pd.StringDtype(),
        "month": pd.Int8Dtype(),
        "visitor_type": pd.StringDtype(),
        'visitors_pct_change': pd.Float32Dtype(),
        'revenue_pct_change': pd.Float32Dtype(),
        'pageviews_pct_change': pd.Float32Dtype(),
        'time_on_site_pct_change': pd.Float32Dtype(),
        'bounce_rate_pct_change': pd.Float32Dtype(),
        'conversion_rate_pct_change': pd.Float32Dtype(),
        'product_clicks_rate_pct_change': pd.Float32Dtype(),
        'add_to_cart_rate_pct_change': pd.Float32Dtype(),
    },
    visitor_type_col='split_type',
    # list of months in chronological order of the visits in the GA360 data (starting
    # in August 2016 and ending in August 2017)
    df_months_ordered=pd.DataFrame(
        [8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8], columns=['month']
    ),
    zero_replacement_dict={"return_purchasers": {0: None}, "conversion_rate": {0: None}},
    cols_to_drop=[
        'add_to_cart',
        'product_views',
        'product_clicks',
        'bounces',
    ],
    mom_stats=[
        'visitors',
        'revenue',
        'pageviews',
        'time_on_site',
        'bounce_rate',
        'conversion_rate',
        'product_clicks_rate',
        'add_to_cart_rate',
    ],
).assign(split_type=lambda df: df["split_type"].str.replace("_", "+").str.title())
for c in (
    df_summary.columns[
        df_summary.columns.str.endswith("_pct_change")
    ].tolist()
):
    df_summary[f"{c}_gt_0"] = (df_summary[c].astype("float") > 0).astype(pd.BooleanDtype())
with pd.option_context('display.max_columns', None):
    display(df_summary)

,month,split_type,return_purchasers,revenue,visitors,pageviews,time_on_site,channelGrouping,deviceCategory,browser,os,audience_strategy,bounce_rate,product_clicks_rate,add_to_cart_rate,visitor_type,conversion_rate,visitors_pct_change,revenue_pct_change,pageviews_pct_change,time_on_site_pct_change,bounce_rate_pct_change,conversion_rate_pct_change,product_clicks_rate_pct_change,add_to_cart_rate_pct_change,visitors_pct_change_gt_0,revenue_pct_change_gt_0,pageviews_pct_change_gt_0,time_on_site_pct_change_gt_0,bounce_rate_pct_change_gt_0,conversion_rate_pct_change_gt_0,product_clicks_rate_pct_change_gt_0,add_to_cart_rate_pct_change_gt_0
0,9,Train+Val,515,25347.169922,18612,122012,3.155372,Organic Search,desktop,Chrome,Windows,1,30.217064,1.754417,20.470665,return_purchasers,2.767032,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,False,False,False,False,False,False,False
1,10,Train+Val,744,56828.398438,22603,137467,3.180688,Organic Search,desktop,Chrome,Macintosh,1,30.67292,2.973817,18.134762,return_purchasers,3.291599,21.443155,124.20018,12.666787,0.802328,1.508602,18.957729,69.504578,-11.410979,True,True,True,True,True,True,True,False
2,11,Train+Val,1651,212099.9375,24400,162391,3.522282,Organic Search,desktop,Chrome,Macintosh,1,27.090164,3.284731,25.233606,return_purchasers,6.766394,7.950272,273.22879,18.130898,10.739615,-11.680517,105.565582,10.455062,39.144966,True,True,True,True,False,True,True,True
3,12,Train+Val,1340,188476.625,26936,168928,3.386973,Organic Search,desktop,Chrome,Macintosh,1,29.287941,3.132498,23.514999,return_purchasers,4.974755,10.393442,-11.137819,4.025469,-3.841523,8.112826,-26.478485,-4.634573,-6.810791,True,False,True,False,True,False,False,False
4,1,Train+Val,722,148537.96875,21177,125278,3.237079,Organic Search,desktop,Chrome,Macintosh,1,33.035839,3.095364,20.805592,return_purchasers,3.409359,-21.380308,-21.190245,-25.839411,-4.425606,12.79673,-31.466791,-1.185452,-11.52204,False,False,False,False,True,False,False,False
5,2,Test,465,110950.46875,20164,106469,2.980083,Organic Search,desktop,Chrome,Macintosh,1,35.226147,2.697536,18.944654,return_purchasers,2.30609,-4.783492,-25.304979,-15.013809,-7.939141,6.630086,-32.360016,-12.852382,-8.944409,False,False,False,False,True,False,False,False
6,3,Infer,<NA>,135771.375,21752,117350,3.199325,Organic Search,desktop,Chrome,Macintosh,1,33.357853,2.919381,19.676352,all_visitors,<NA>,7.875422,22.371161,10.219876,7.356926,-5.303712,0.0,8.223991,3.862292,True,True,True,True,False,False,True,True


### Categorical Feature KPIs

In [39]:
#| echo: false
#| output: true
infer_month = df_cohorts['month'].unique().tolist()[0]
df_development_grouped = df_development.query(f"month=={infer_month-1}").copy()
dfs_development_agg = []
for f in categorical_features[1:]:
    if f in ['source', 'browser']:
        df_development_grouped[f] = (
            trh.group_infrequent_categories(
                df_development_grouped[f], f
            )
        )
    df_development_agg = trh.agg_kpis(df_development_grouped, f)
    dfs_development_agg.append(df_development_agg)
df_development_agg = (
    pd.concat(dfs_development_agg, ignore_index=True)
    .astype(
        {
            "feature_name": pd.StringDtype(),
            "feature_category": pd.StringDtype(),
            "conversions": pd.Int32Dtype(),
            'product_views': pd.Int32Dtype(),
            'product_clicks': pd.Int32Dtype(),
            "visitors": pd.Int32Dtype(),
            "proportion": pd.Float32Dtype(),
            "ctr": pd.Float32Dtype(),
            "conversion_rate": pd.Float32Dtype(),
            "feature": pd.StringDtype(),
        }
    )
    .melt(
        id_vars=[
            'feature_name',
            'feature_category',
        ],
        value_vars=[
            'ctr',
            'conversion_rate',
            'revenue',
            'conversions',
            'product_views',
            'product_clicks',
            'visitors',
            'proportion',
        ]
    )
    .assign(
        variable=lambda df: (
            df['variable'].str.replace('_', ' ')
            .str.title().str.replace('Ctr', 'CTR')
        )
    )
    .assign(audience_strategy=audience_strategy)
    .assign(historical_data_month=month_name[infer_month-1])
    .assign(historical_data_size=len(df_development_grouped))
    .astype(
        {
            "historical_data_month": pd.StringDtype(),
            "historical_data_size": pd.Int32Dtype(),
            'audience_strategy': pd.Int8Dtype(),
            "variable": pd.StringDtype(),
            "value": pd.Float32Dtype(),
        }
    )
)
with pd.option_context('display.max_rows', None):
    display(
        df_development_agg.query("variable.str.contains('CTR|Conversion Rate')")
        .pivot(
            index=[
                'audience_strategy',
                'historical_data_month',
                'historical_data_size',
                'feature_name',
                'feature_category',
            ],
            columns=['variable'],
            values=['value'],
        )
        .reset_index()
    )

audience_strategy historical_data_month historical_data_size  \
variable                                                                
0                        1              February                20164   
1                        1              February                20164   
2                        1              February                20164   
3                        1              February                20164   
4                        1              February                20164   
5                        1              February                20164   
6                        1              February                20164   
7                        1              February                20164   
8                        1              February                20164   
9                        1              February                20164   
10                       1              February                20164   
11                       1              February                20164   
12                       1              February                20164   
13                       1              February                20164   
14                       1              February                20164   
15                       1              February                20164   
16                       1              February                20164   
17                       1              February                20164   
18                       1              February                20164   
19                       1              February                20164   
20                       1              February                20164   
21                       1              February                20164   
22                       1              February                20164   
23                       1              February                20164   
24                       1              February                20164   
25                       1              February                20164   
26                       1              February                20164   
27                       1              February                20164   
28                       1              February                20164   
29                       1              February                20164   
30                       1              February                20164   
31                       1              February                20164   
32                       1              February                20164   
33                       1              February                20164   
34                       1              February                20164   
35                       1              February                20164   
36                       1              February                20164   
37                       1              February                20164   

             feature_name                feature_category           value  \
variable                                                  CTR               
0                 browser                          Chrome        2.878088   
1                 browser                          Safari        2.060734   
2                 browser                           other         2.30855   
3         channelGrouping                      Affiliates        1.399795   
4         channelGrouping                          Direct        3.610331   
5         channelGrouping                         Display        2.531646   
6         channelGrouping                  Organic Search        2.342916   
7         channelGrouping                     Paid Search        2.604475   
8         channelGrouping                        Referral        3.109836   
9         channelGrouping                          Social        2.114267   
10         deviceCategory                         desktop        3.008399   
11         deviceCategory                          mobile        1.890257   
12        

## Upload to BigQuery Tables

### Audience Feature Importances

::: {.content-hidden}
Show summary DataFrame with feature importances (see the second item within #1. from the **About** section above)
:::

In [40]:
#| output: false
summarize_df(df_feats)

,column,dtype,missing
0,audience_strategy,Int8,0
1,num_observations,Int16,0
2,stat,string[python],0
3,maudience,string[python],0
4,value,Float32,0


Define BigQuery Table Schema

In [41]:
#| echo: true
job_config_feats_imp = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("num_observations", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("stat", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("maudience", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("value", "FLOAT64", mode='NULLABLE'),
    ]
)
job_config_feats_imp.write_disposition = 'WRITE_APPEND'

Create BigQuery table (if it does not exist) and populate

In [42]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_table_fully_resolved_feats_imp, client)
bquh.append_df_to_bq_table(
    df_feats, job_config_feats_imp, gbq_table_fully_resolved_feats_imp, client
)

Created table named demoabc-381618.mydemo2asdf.audience_feats_imp
Completed upload
Found 30 rows and 5 columns in table mydemo2asdf.audience_feats_imp


### Audience Profile

::: {.content-hidden}
Show summary DataFrame with audience profile (see the first item within #1. from the **About** section above)
:::

In [43]:
#| output: false
summarize_df(df_profile_sliced)

,column,dtype,missing
0,Audience_Strategy,Int8,0
1,Stat_Expanded,string[python],0
2,High,Float32,0
3,Medium,Float32,0
4,Low,Float32,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [44]:
#| echo: true
job_config_profiles = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("Audience_Strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("Stat_Expanded", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("High", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("Medium", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("Low", "FLOAT64", mode='NULLABLE'),
    ]
)
job_config_profiles.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [45]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_table_fully_resolved_profiles, client)
bquh.append_df_to_bq_table(
    df_profile_sliced, job_config_profiles, gbq_table_fully_resolved_profiles, client
)

Created table named demoabc-381618.mydemo2asdf.audience_profiles
Completed upload
Found 20 rows and 5 columns in table mydemo2asdf.audience_profiles


### Audience Cohorts

::: {.content-hidden}
Show summary `DataFrame` with inference data and predicted audience cohorts (see #2. from the **About** section above)
:::

In [46]:
#| output: false
summarize_df(df_dev_cohorts)

,column,dtype,missing
0,infer_month,string[python],133892
1,fullvisitorid,string[python],0
2,visitId,string[python],0
3,visitNumber,Int8,0
4,visitStartTime,datetime64[ns],0
5,quarter,Int8,0
6,month,Int8,0
7,day_of_month,Int8,0
8,day_of_week,Int8,0
9,hour,Int8,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [47]:
#| echo: true
job_config_cohorts = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("infer_month", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("fullvisitorid", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("visitId", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("visitNumber", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("visitStartTime", "DATETIME", mode='NULLABLE'),
        bigquery.SchemaField("quarter", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("month", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("day_of_month", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("day_of_week", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("hour", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("minute", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("second", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("source", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("medium", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("channelGrouping", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("hits", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("bounces", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("last_action", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("promos_displayed", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("promos_clicked", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("product_views", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("product_clicks", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("pageviews", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("time_on_site", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("browser", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("os", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("deviceCategory", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("added_to_cart", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("revenue", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("score", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("predicted_score_label", "BOOLEAN", mode='NULLABLE'),
        bigquery.SchemaField("maudience", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("cohort", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("made_purchase_on_future_visit", "BOOLEAN", mode='NULLABLE'),
        bigquery.SchemaField("split_type", "STRING", mode='NULLABLE'),
    ]
)
job_config_cohorts.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [48]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_table_fully_resolved_cohorts, client)
bquh.append_df_to_bq_table(
    df_dev_cohorts, job_config_cohorts, gbq_table_fully_resolved_cohorts, client
)

Created table named demoabc-381618.mydemo2asdf.audience_cohorts
Completed upload
Found 155,644 rows and 36 columns in table mydemo2asdf.audience_cohorts


### Monthly Performance Summary

::: {.content-hidden}
Show summary `DataFrame` with monthly summary statistics for the inference data (see #3. from the **About** section above)
:::

In [49]:
#| output: false
summarize_df(df_summary)

,column,dtype,missing
0,month,Int8,0
1,split_type,string[python],0
2,return_purchasers,Int16,1
3,revenue,Float32,0
4,visitors,Int16,0
5,pageviews,Int32,0
6,time_on_site,Float32,0
7,channelGrouping,string[python],0
8,deviceCategory,string[python],0
9,browser,string[python],0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [50]:
#| echo: true
job_config_summary = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("month", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("split_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("return_purchasers", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("revenue", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("visitors", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("pageviews", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("time_on_site", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("channelGrouping", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("deviceCategory", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("browser", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("os", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("bounce_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("product_clicks_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("add_to_cart_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("visitor_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("conversion_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("visitors_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("revenue_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("pageviews_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("time_on_site_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("bounce_rate_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("conversion_rate_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("product_clicks_rate_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("add_to_cart_rate_pct_change", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("visitors_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("revenue_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("pageviews_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("time_on_site_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("bounce_rate_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("conversion_rate_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("product_clicks_rate_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
        bigquery.SchemaField("add_to_cart_rate_pct_change_gt_0", "BOOLEAN", mode='REQUIRED'),
    ]
)
job_config_summary.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [51]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_summary_table_id_fully_resolved, client)
bquh.append_df_to_bq_table(
    df_summary, job_config_summary, gbq_summary_table_id_fully_resolved, client
)

Created table named demoabc-381618.mydemo2asdf.monthly_summary
Completed upload
Found 7 rows and 33 columns in table mydemo2asdf.monthly_summary


### Conversion Rates in Development and Inference

::: {.content-hidden}
Show `DataFrame` with conversion rates (see #4. from the **About** section above)
:::

In [52]:
#| output: false
summarize_df(df_conv_rates)

,column,dtype,missing
0,audience_strategy,Int8,0
1,infer_month,string[python],3
2,maudience,string[python],0
3,pred_conversions,Int16,0
4,total_visitors,Int16,0
5,min_score,Float32,0
6,true_conversions,Int16,0
7,data_type,string[python],0
8,data_size,Int16,0
9,true_conv_rate,Float32,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [53]:
#| echo: true
job_config_conv_rates = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("infer_month", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("maudience", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("pred_conversions", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("total_visitors", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("min_score", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("true_conversions", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("data_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("data_size", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("true_conv_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("overall_true_conv_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("pred_conv_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("overall_pred_conv_rate", "FLOAT64", mode='NULLABLE'),
    ]
)
job_config_conv_rates.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [54]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_conv_rates_table_id_fully_resolved, client)
bquh.append_df_to_bq_table(
    df_conv_rates, job_config_conv_rates, gbq_conv_rates_table_id_fully_resolved, client
)

Created table named demoabc-381618.mydemo2asdf.audience_conversion_rates
Completed upload
Found 6 rows and 13 columns in table mydemo2asdf.audience_conversion_rates


### Estimated and Actual Cohort to Audience Fractions

::: {.content-hidden}
Show `DataFrame` with cohort-to-audience fractions (see #5. from the **About** section above)
:::

In [55]:
#| output: false
summarize_df(df_sa_frac)

,column,dtype,missing
0,audience_strategy,Int8,0
1,infer_month,string[python],6
2,maudience,string[python],0
3,cohort,string[python],0
4,size,Int16,0
5,group_size,Int16,0
6,uplift,Int8,0
7,power,Int8,0
8,ci_level,Int8,0
9,samp_to_aud_frac,Float32,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [56]:
#| echo: true
job_config_sa_frac = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("infer_month", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("maudience", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("cohort", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("size", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("group_size", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("uplift", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("power", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("ci_level", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("samp_to_aud_frac", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("size_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("data_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("data_size", "INTEGER", mode='NULLABLE'),
    ]
)
job_config_sa_frac.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [57]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_sa_fracs_table_id_fully_resolved, client)
bquh.append_df_to_bq_table(
    df_sa_frac, job_config_sa_frac, gbq_sa_fracs_table_id_fully_resolved, client
)

Created table named demoabc-381618.mydemo2asdf.cohort_audience_fractions
Completed upload
Found 12 rows and 13 columns in table mydemo2asdf.cohort_audience_fractions


### Aggregated Conversion Rates

::: {.content-hidden}
Show `DataFrame` with combined aggregated conversion rates (see #6. from the **About** section above)
:::

In [58]:
#| output: false
summarize_df(df_conv_rates_agg_combo)

,column,dtype,missing
0,maudience,string[python],6
1,data_type,string[python],0
2,var,string[python],0
3,value,Float32,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [59]:
#| echo: true
job_config_conv_rates_combo = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("maudience", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("data_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("var", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("value", "FLOAT64", mode='NULLABLE'),
    ]
)
job_config_conv_rates_combo.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [60]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_conv_rates_combo_table_id_fully_resolved, client)
bquh.append_df_to_bq_table(
    df_conv_rates_agg_combo,
    job_config_conv_rates_combo,
    gbq_conv_rates_combo_table_id_fully_resolved,
    client
)

Created table named demoabc-381618.mydemo2asdf.conversion_rates_aggregated
Completed upload
Found 18 rows and 4 columns in table mydemo2asdf.conversion_rates_aggregated


### Daily Performance Summary

::: {.content-hidden}
Show `DataFrame` with combined daily performance summary (see #7. from the **About** section above)
:::

In [61]:
#| output: false
summarize_df(df_hmap_combo)

,column,dtype,missing
0,month,string[python],0
1,date,string[python],0
2,maudience,string[python],0
3,return_purchasers,Int16,0
4,revenue,Float32,0
5,visitors,Int16,0
6,add_to_cart,Int32,0
7,pageviews,Int32,0
8,time_on_site,Float32,0
9,product_views,Int32,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [62]:
#| echo: true
job_config_daily_perf_combo = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("month", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("date", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("maudience", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("return_purchasers", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("revenue", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("visitors", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("add_to_cart", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("pageviews", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("time_on_site", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("product_views", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("product_clicks", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("bounces", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("bounce_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("product_clicks_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("add_to_cart_rate", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("visitor_type", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("agg_type", "STRING", mode='NULLABLE'),
    ]
)
job_config_daily_perf_combo.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [63]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_daily_perf_combo_table_id_fully_resolved, client)
bquh.append_df_to_bq_table(
    df_hmap_combo,
    job_config_daily_perf_combo,
    gbq_daily_perf_combo_table_id_fully_resolved,
    client
)

Created table named demoabc-381618.mydemo2asdf.daily_summary
Completed upload
Found 486 rows and 18 columns in table mydemo2asdf.daily_summary


### Categorical Feature KPIs

::: {.content-hidden}
Show summary DataFrame with categorical feature KPIs (see the second item within #1. from the **About** section above)
:::

In [64]:
#| output: false
summarize_df(df_development_agg)

,column,dtype,missing
0,feature_name,string[python],0
1,feature_category,string[python],0
2,variable,string[python],0
3,value,Float32,0
4,audience_strategy,Int8,0
5,historical_data_month,string[python],0
6,historical_data_size,Int32,0


::: {.content-hidden}
Define BigQuery Table Schema
:::

In [65]:
#| echo: true
job_config_cat_feats_kpis = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("feature_name", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("feature_category", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("variable", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("value", "FLOAT64", mode='NULLABLE'),
        bigquery.SchemaField("audience_strategy", "INTEGER", mode='NULLABLE'),
        bigquery.SchemaField("historical_data_month", "STRING", mode='NULLABLE'),
        bigquery.SchemaField("historical_data_size", "INTEGER", mode='NULLABLE'),
    ]
)
job_config_cat_feats_kpis.write_disposition = 'WRITE_APPEND'

::: {.content-hidden}
Create BigQuery table (if it does not exist) and populate
:::

In [66]:
#| echo: true
#| output: false
bquh.create_bq_table(gbq_table_fully_resolved_cat_feat_kpis, client)
bquh.append_df_to_bq_table(
    df_development_agg,
    job_config_cat_feats_kpis,
    gbq_table_fully_resolved_cat_feat_kpis,
    client,
)

Created table named demoabc-381618.mydemo2asdf.categorical_features_kpis
Completed upload
Found 304 rows and 7 columns in table mydemo2asdf.categorical_features_kpis


## Next Step

The next step will be to create summary charts for an end-user using the contents of these two newly created audience tables.